In [1]:
import importlib.util
import matplotlib.pyplot as plt
from matplotlib.pyplot import xticks
import numpy as np
import pandas as pd
import os
import time

In [2]:
class Recap():
    
    def __init__(self):
        # Filepath to Excel raw file
        # self.origin_file = "./raw/Activities_A_test.xlsx"
        self.origin_file = "./raw/Activities_A.xlsx"
        
        # Filepath to Excel treated file
        self.target_file = "./treated/Activities_A.xlsx"
        
        # Start_time
        self.start_time = "16:05:00"
        
        # Raw Data
        self.raw_data = None
        self.get_raw_data()
        
        # Treated Data
        self.treated_data = None
        
        # All submissions list
        self.all_subs_list = []
        self.get_all_subs()
        
        # Grade submissions
        self.grades_list = []
        self.get_all_grades()
        
        # Save to file
        self.recreate_pd_data()
        self.save_to_excel()
        
        
    def get_raw_data(self):
        
        # Read raw data from file
        self.raw_data = pd.read_excel(self.origin_file).astype(str).to_numpy()
        
    def get_all_subs(self):
        
        for submission in self.raw_data:
            self.all_subs_list.append(Submission(submission))
            
    def get_all_grades(self):
        
        for submission in self.all_subs_list:
            self.grades_list.append((submission.name, submission.ID, submission.total_grade))
        
        self.unique_grades_list = []
        for grade in self.grades_list:
            ID_list = [tup[1] for tup in self.unique_grades_list]
            if not grade[1] in ID_list:
                self.unique_grades_list.append(grade)
            else:
                print("Duplicate found for: ", grade[1])
                
    def recreate_pd_data(self):
        
        # Initialize dictionary
        data_dic = {"Name": [], "ID": [], "Time to submit": []}
        data_dic.update({"Q{}".format(i+1): [] for i in range(5)})
        data_dic.update({"Total Activities score": []})
        
        # Add entries
        for submission in self.all_subs_list:
            data_dic["Name"].append(submission.name)
            data_dic["ID"].append(submission.ID)
            data_dic["Time to submit"].append(submission.elapsed_time)
            for index, score in enumerate(submission.scores_list):
                data_dic["Q{}".format(index + 1)].append(score)
            data_dic["Total Activities score"].append(submission.total_grade)
        
        # To pandas
        self.treated_data = pd.DataFrame(data_dic)
        
    
    def save_to_excel(self):
        
        self.treated_data.to_excel(self.target_file, engine = "xlsxwriter") 

In [3]:
class Question():
    
    def __init__(self, number, points, f_as_str):
        # Question number
        self.number = number
        
        # Number of points
        self.points = points
        
        # Function as string
        self.f_as_str = f_as_str
        
        
    def str_to_file(self, filepath, filename):
        # Send file in string format to file
        path_to_file = filepath + filename
        if not os.path.exists(filepath):
            os.makedirs(filepath)
        with open(path_to_file, "w") as file:
            file.write(self.f_as_str)
            file.close()
        self.path_to_file = path_to_file

In [4]:
class Question1(Question):
    
    def __init__(self, f_as_str):
        # Reusing mother class init
        super().__init__(number = 1, points = 10, f_as_str = f_as_str)
        
        
    def import_from_file(self, filepath, filename):
        try:
            # Import function from file
            self.str_to_file(filepath, filename)
            spec = importlib.util.spec_from_file_location("convert_hours", self.path_to_file)
            function = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(function)
            self.function_name = getattr(function, "convert_hours")
            self.malfunctioning = False
        except:
            self.malfunctioning = True
        
    
    def grade(self):
        # Grade function by test cases
        test_cases_number = 4
        total_points = 0
        points_per_tc = self.points/test_cases_number
        print("-----")
        if(self.malfunctioning):
            self.override = True
            print("Could not import function. Displaying function below. \n")
            print(self.f_as_str, end = "\n\n")
            total_points = -1
            while(total_points < 0 or total_points > self.points):
                time.sleep(1)
                total_points = float(input("How many points? (max. {}): ".format(self.points)))
            return total_points
        else:
            try:
                weeks, days, hours = self.function_name(257)
                total_points += points_per_tc*int(weeks == 1 and days == 3 and hours == 17)
                weeks, days, hours = self.function_name(1024)
                total_points += points_per_tc*int(weeks == 6 and days == 0 and hours == 16)
                weeks, days, hours = self.function_name(34)
                total_points += points_per_tc*int(weeks == 0 and days == 1 and hours == 10)
                weeks, days, hours = self.function_name(3)
                total_points += points_per_tc*int(weeks == 0 and days == 0 and hours == 3)
                self.override = False
                print("Testing function below. \n")
                print(self.f_as_str, end = "\n\n")
                print("Function was given {} / {} by autograder".format(total_points, self.points))
                assert total_points == self.points, "Did not obtain maximal score."
                return total_points
            except:
                self.override = True
                print("Error detected in function. Displaying function below. \n")
                print(self.f_as_str, end = "\n\n")
                total_points = -1
                while(total_points < 0 or total_points > self.points):
                    time.sleep(1)
                    total_points = float(input("How many points? (max. {}): ".format(self.points)))
                return total_points

In [5]:
class Question2(Question):
    
    def __init__(self, f_as_str):
        # Reusing mother class init
        super().__init__(number = 2, points = 10, f_as_str = f_as_str)
        
        
    def import_from_file(self, filepath, filename):
        try:
            # Import function from file
            self.str_to_file(filepath, filename)
            spec = importlib.util.spec_from_file_location("is_leap_year", self.path_to_file)
            function = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(function)
            self.function_name = getattr(function, "is_leap_year")
            self.malfunctioning = False
        except:
            self.malfunctioning = True
        
    
    def grade(self):
        # Grade function by test cases
        test_cases_number = 4
        total_points = 0
        points_per_tc = self.points/test_cases_number
        print("-----")
        if(self.malfunctioning):
            self.override = True
            print("Could not import function. Displaying function below. \n")
            print(self.f_as_str, end = "\n\n")
            total_points = -1
            while(total_points < 0 or total_points > self.points):
                time.sleep(1)
                total_points = float(input("How many points? (max. {}): ".format(self.points)))
            return total_points
        else:
            try:
                res = self.function_name(2003)
                total_points += points_per_tc*int(res == False)
                res = self.function_name(2004)
                total_points += points_per_tc*int(res == True)
                res = self.function_name(1800)
                total_points += points_per_tc*int(res == False)
                res = self.function_name(1600)
                total_points += points_per_tc*int(res == True)
                self.override = False
                print("Testing function below. \n")
                print(self.f_as_str, end = "\n\n")
                print("Function was given {} / {} by autograder".format(total_points, self.points))
                assert total_points == self.points, "Did not obtain maximal score."
                return total_points
            except:
                self.override = True
                print("Error detected in function. Displaying function below. \n")
                print(self.f_as_str, end = "\n\n")
                total_points = -1
                while(total_points < 0 or total_points > self.points):
                    time.sleep(1)
                    total_points = float(input("How many points? (max. {}): ".format(self.points)))
                return total_points

In [6]:
class Question3(Question):
    
    def __init__(self, f_as_str):
        # Reusing mother class init
        super().__init__(number = 3, points = 20, f_as_str = f_as_str)
        
        
    def import_from_file(self, filepath, filename):
        try:
            # Import function from file
            self.str_to_file(filepath, filename)
            spec = importlib.util.spec_from_file_location("how_many_divide", self.path_to_file)
            function = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(function)
            self.function_name = getattr(function, "how_many_divide")
            self.malfunctioning = False
        except:
            self.malfunctioning = True
        
    
    def grade(self):
        # Grade function by test cases
        test_cases_number = 4
        total_points = 0
        points_per_tc = self.points/test_cases_number
        print("-----")
        if(self.malfunctioning):
            self.override = True
            print("Could not import function. Displaying function below. \n")
            print(self.f_as_str, end = "\n\n")
            total_points = -1
            while(total_points < 0 or total_points > self.points):
                time.sleep(1)
                total_points = float(input("How many points? (max. {}): ".format(self.points)))
            return total_points
        else:
            try:
                res = self.function_name(8)
                total_points += points_per_tc*int(res == 3)
                res = self.function_name(72)
                total_points += points_per_tc*int(res == 3)
                res = self.function_name(100)
                total_points += points_per_tc*int(res == 2)
                res = self.function_name(5)
                total_points += points_per_tc*int(res == 0)
                self.override = False
                print("Testing function below. \n")
                print(self.f_as_str, end = "\n\n")
                print("Function was given {} / {} by autograder".format(total_points, self.points))
                assert total_points == self.points, "Did not obtain maximal score."
                return total_points
            except:
                self.override = True
                print("Error detected in function. Displaying function below. \n")
                print(self.f_as_str, end = "\n\n")
                total_points = -1
                while(total_points < 0 or total_points > self.points):
                    time.sleep(1)
                    total_points = float(input("How many points? (max. {}): ".format(self.points)))
                return total_points

In [7]:
class Question4(Question):
    
    def __init__(self, f_as_str):
        # Reusing mother class init
        super().__init__(number = 4, points = 30, f_as_str = f_as_str)
        
        
    def import_from_file(self, filepath, filename):
        try:
            # Import function from file
            self.str_to_file(filepath, filename)
            spec = importlib.util.spec_from_file_location("find_if_triplet", self.path_to_file)
            function = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(function)
            self.function_name = getattr(function, "find_if_triplet")
            self.malfunctioning = False
        except:
            self.malfunctioning = True
        
    
    def grade(self):
        # Grade function by test cases
        test_cases_number = 4
        total_points = 0
        points_per_tc = self.points/test_cases_number
        print("-----")
        if(self.malfunctioning):
            self.override = True
            print("Could not import function. Displaying function below. \n")
            print(self.f_as_str, end = "\n\n")
            total_points = -1
            while(total_points < 0 or total_points > self.points):
                time.sleep(1)
                total_points = float(input("How many points? (max. {}): ".format(self.points)))
            return total_points
        else:
            try:
                res = self.function_name([7,7,7])
                total_points += points_per_tc*int(res == True)
                res = self.function_name([1,2,2,2])
                total_points += points_per_tc*int(res == True)
                res = self.function_name([1,2,2,3,3,3,4,4])
                total_points += points_per_tc*int(res == True)
                res = self.function_name([4,5,5,6,7,7,8,9,9])
                total_points += points_per_tc*int(res == False)
                self.override = False
                print("Testing function below. \n")
                print(self.f_as_str, end = "\n\n")
                print("Function was given {} / {} by autograder".format(total_points, self.points))
                assert total_points == self.points, "Did not obtain maximal score."
                return total_points
            except:
                self.override = True
                print("Error detected in function. Displaying function below. \n")
                print(self.f_as_str, end = "\n\n")
                total_points = -1
                while(total_points < 0 or total_points > self.points):
                    time.sleep(1)
                    total_points = float(input("How many points? (max. {}): ".format(self.points)))
                return total_points

In [8]:
class Question5(Question):
    
    def __init__(self, f_as_str):
        # Reusing mother class init
        super().__init__(number = 5, points = 30, f_as_str = f_as_str)
        
        
    def import_from_file(self, filepath, filename):
        try:
            # Import function from file
            self.str_to_file(filepath, filename)
            spec = importlib.util.spec_from_file_location("is_sum_squares", self.path_to_file)
            function = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(function)
            self.function_name = getattr(function, "is_sum_squares")
            self.malfunctioning = False
        except:
            self.malfunctioning = True
        
    
    def grade(self):
        # Grade function by test cases
        test_cases_number = 4
        total_points = 0
        points_per_tc = self.points/test_cases_number
        print("-----")
        if(self.malfunctioning):
            self.override = True
            print("Could not import function. Displaying function below. \n")
            print(self.f_as_str, end = "\n\n")
            total_points = -1
            while(total_points < 0 or total_points > self.points):
                time.sleep(1)
                total_points = float(input("How many points? (max. {}): ".format(self.points)))
            return total_points
        else:
            try:
                res = self.function_name(16)
                total_points += points_per_tc*int(res == True)
                res = self.function_name(32)
                total_points += points_per_tc*int(res == True)
                res = self.function_name(48)
                total_points += points_per_tc*int(res == False)
                res = self.function_name(52)
                total_points += points_per_tc*int(res == True)
                self.override = False
                print("Testing function below. \n")
                print(self.f_as_str, end = "\n\n")
                print("Function was given {} / {} by autograder".format(total_points, self.points))
                assert total_points == self.points, "Did not obtain maximal score."
                return total_points
            except:
                self.override = True
                print("Error detected in function. Displaying function below. \n")
                print(self.f_as_str, end = "\n\n")
                total_points = -1
                while(total_points < 0 or total_points > self.points):
                    time.sleep(1)
                    total_points = float(input("How many points? (max. {}): ".format(self.points)))
                return total_points

In [9]:
class Submission():
    
    def __init__(self, student_sub):
        # Student name
        self.name = None
        self.get_name(student_sub)
        
        # Student ID
        self.ID = None
        self.get_ID(student_sub)
        
        # Timestamp
        self.timestamp = None
        self.convert_timestamp(student_sub)
        
        # Elapsed time
        self.elapsed_time = None
        
        # Answers_list
        self.answers_list = []
        print("--------------------")
        print("Checking submission for student {}, with ID {}.".format(self.name, self.ID))
        self.get_answers_list(student_sub)
        
        # Scores list
        self.scores_list = []
        
        # Total grade
        self.total_grade = 0
        self.grade_questions()
        
        
    def get_name(self, student_sub):
        self.name = student_sub[1]
        
        
    def get_ID(self, student_sub):
        self.ID = student_sub[2]
        
        
    def convert_timestamp(self, student_sub):
        self.timestamp = student_sub[0][-12:-4]
        
        
    def get_answers_list(self, student_sub):
        # Recreate questions objects with student submissions
        self.answers_list = []
        filepath = "./functions/" + self.ID + "/"
        # Q1
        q1 = Question1(student_sub[3])
        q1.import_from_file(filepath, "convert_hours.py")
        self.answers_list.append(q1)
        # Q2
        q2 = Question2(student_sub[4])
        q2.import_from_file(filepath, "is_leap_year.py")
        self.answers_list.append(q2)
        # Q3
        q3 = Question3(student_sub[5])
        q3.import_from_file(filepath, "how_many_divide.py")
        self.answers_list.append(q3)
        # Q4
        q4 = Question4(student_sub[6])
        q4.import_from_file(filepath, "find_if_triplet.py")
        self.answers_list.append(q4)
        # Q5
        q5 = Question5(student_sub[7])
        q5.import_from_file(filepath, "is_sum_squares.py")
        self.answers_list.append(q5)
        
        
    def grade_questions(self):
        # Grade submissions and compute score for this submission
        for question in self.answers_list:
            self.scores_list.append(question.grade())
        self.total_grade += self.scores_list[0]
        self.total_grade += self.scores_list[1]
        self.total_grade += self.scores_list[2]
        self.total_grade += self.scores_list[3]
        self.total_grade += self.scores_list[4]

In [10]:
r = Recap()
print(r.__dict__.keys())

--------------------
Checking submission for student Jaron Ho, with ID 1005011.
-----
Testing function below. 

def convert_hours(total_hours):
    weeks = total_hours // (24*7)
    days = total_hours // 24 - weeks * 7
    hours = total_hours - weeks * 24 * 7 - days * 24
    return weeks, days, hours

Function was given 10.0 / 10 by autograder
-----
Testing function below. 

def is_leap_year(year):
    is_ly = (year % 400 == 0) or ((year % 4 == 0) and (year % 25 != 0))
    return is_ly

Function was given 10.0 / 10 by autograder
-----
Testing function below. 

def how_many_divide(number):
    counter = 0
    while number >= 2:
        number /= 2
        if (number % 1 != 0):
            break
        counter += 1
    return counter

Function was given 20.0 / 20 by autograder
-----
Testing function below. 

def find_if_triplet(my_list):
    has_triplet = False
    for index, value in enumerate(my_list[:-2]):
        if(value == my_list[index + 1] == my_list[index + 2]):
            has

Testing function below. 

def is_sum_squares(number):
    squared_list = []
    counter = 0;
    while(counter < 10000):
        squared_list.append(counter*counter)
        counter += 1
        
    for a in squared_list:
        for b in squared_list:
            if(a+b == number):
                return True
    return False
    
print(is_sum_squares(9))
print(is_sum_squares(5))
print(is_sum_squares(50))
print(is_sum_squares(7))

Function was given 30.0 / 30 by autograder
--------------------
Checking submission for student Ryan Toh, with ID 1005129.
-----
Testing function below. 

def convert_hours(total_hours):
    weeks = total_hours//24//7
    days = total_hours//24 - (weeks*7)
    hours = total_hours - (weeks*7*24) - (days*24)
    return weeks, days, hours

Function was given 10.0 / 10 by autograder
-----
Testing function below. 

def is_leap_year(year):
    is_ly = ((year % 4 == 0) and (year % 100 != 0)) or ((year % 4 == 0) and (year % 400 == 0))
    return is_ly

Function was

How many points? (max. 30): 20
--------------------
Checking submission for student Aditya Sudarshan Rajesh Krishnan, with ID 1005395.
-----
weeks =0, days = 1, hours = 3
weeks =0, days = 6, hours = 35
weeks =0, days = 1, hours = 10
weeks =0, days =0, hours = 3
Testing function below. 

def convert_hours(total_hours):
    weeks = None
    days = None
    hours = None
    if total_hours < 24 and total_hours >= 0:
        print("weeks =0, days =0, hours = {}".format(total_hours))
    elif total_hours >=24 and total_hours < 168:
        days = total_hours//24
        #print(days)
        hours = total_hours - 24*days 
        #print(hours)
        print("weeks =0, days = {}, hours = {}".format(days,hours))
    elif total_hours >= 168:
        weeks = total_hours//168
        #print(weeks)
        days = (total_hours - 168)//24
        #print(days)
        hours = (total_hours - 168) - days*24
        print("weeks =0, days = {}, hours = {}".format(weeks,days,hours))
    return weeks, days,

How many points? (max. 30): 22.5
-----
Testing function below. 

def is_sum_squares(number):
    is_sq = False
    
    for x in range(number+1):
        for y in range(number+1):
            if number == (x)**2 + (y)**2:
                is_sq = True
    
    return is_sq

Function was given 30.0 / 30 by autograder
--------------------
Checking submission for student Lek Jie Wei, with ID 1005007.
-----
Testing function below. 

def convert_hours(total_hours):
    weeks = 0
    days = 0
    hours = 0
    
    while total_hours > 168 :
        total_hours = total_hours - 168
        weeks = weeks + 1
    while total_hours > 24 :
        total_hours = total_hours - 24
        days = days + 1
    hours = total_hours
    
    return weeks, days, hours

Function was given 10.0 / 10 by autograder
-----
Testing function below. 

def is_leap_year(year):
    
    if year % 100 == 0 :
        is_ly = year % 400 == 0
        
    else:
        is_ly = year % 4 == 0


    return is_ly

Function was

Testing function below. 

def is_sum_squares(number):
    
    is_sq = False
    
    square = [a**2 for a in range (0,9000,1)]
    
    if number in square:
        is_sq = True

    for b in square:
        number2 = number - b 

        if number2 in square:
            is_sq = True
            break

        else:
            is_sq = False
                
            
    return is_sq

Function was given 30.0 / 30 by autograder
--------------------
Checking submission for student Lee Chang Zheng, with ID 1005289.
(0, 0, 7)
(0, 2, 7)
(1, 3, 7)
True
False
False
True
0
2
2
6
True
True
True
False
True
True
True
False
-----
Testing function below. 

def convert_hours(total_hours):
    weeks = total_hours//168
    days = (total_hours - weeks*168)//24
    hours = total_hours - weeks*168 - days*24
    return weeks, days, hours

print(convert_hours(7))
print(convert_hours(55))
print(convert_hours(247))

Function was given 10.0 / 10 by autograder
-----
Testing function below. 

def is_leap_

...

How many points? (max. 30): 0
--------------------
Checking submission for student Lin Lin, with ID 1005275.
False
False
False
True
1
2
6
32
True
False
False
False
-----
Could not import function. Displaying function below. 

def convert_hours(total_hours):
    weeks = int(total_hours/24/7)
    days = int(total_hours/24)
    hours = total_hours%
    return weeks, days, hours

print(convert_hours(7))
print(convert_hours(55))
print(convert_hours(247))

How many points? (max. 10): 2.5
-----
Testing function below. 

def is_leap_year(year):
    is_ly = bool (year%400==0)
    return is_ly

print(is_leap_year(1996))
print(is_leap_year(1997))
print(is_leap_year(1700))
print(is_leap_year(2000))

Function was given 7.5 / 10 by autograder
Error detected in function. Displaying function below. 

def is_leap_year(year):
    is_ly = bool (year%400==0)
    return is_ly

print(is_leap_year(1996))
print(is_leap_year(1997))
print(is_leap_year(1700))
print(is_leap_year(2000))

How many points? (max. 10)

How many points? (max. 30): 0
--------------------
Checking submission for student Koh Wen Xin, with ID 1005426.
-----
Testing function below. 

def convert_hours(total_hours):
    if(total_hours>=168):
        weeks = int(total_hours/168)
        days = int((total_hours - weeks*168)/24)
        hours = int(total_hours - weeks*168 - days*24)
    elif(total_hours>=24):
        weeks = 0
        days = int(total_hours/24)
        hours = int(total_hours - days*24)
    elif(total_hours<24):
        weeks = 0
        days = 0
        hours = int(total_hours)
    return weeks, days, hours

Function was given 10.0 / 10 by autograder
-----
Testing function below. 

def is_leap_year(year):
    is_ly = None
    if(year%4 == 0 and year%100 != 0):
        is_ly = True
    elif(year%4 == 0 and year%100 == 0 and year%400 != 0):
        is_ly = False
    elif(year%4 == 0 and year%100 == 0 and year%400 == 0):
        is_ly = True  
    elif(year%4 != 0):
        is_ly = False
    return is_ly

Functi

How many points? (max. 10): 10
-----
Testing function below. 

def is_leap_year(year):
    is_ly = False
    if (year % 100 == 0):
        if year % 400 == 0:
            is_ly = True
    elif ( year % 4 == 0):
        is_ly = True
    return is_ly 

Function was given 10.0 / 10 by autograder
-----
Could not import function. Displaying function below. 

def how_many_divide(number):
    counter = 0
    if number <> 1:
        while number <= 2 or number % 2 <> 0:
            number = number / 2
            counter += 1
    return counter

How many points? (max. 20): 10
-----
Testing function below. 

def find_if_triplet(my_list):
    has_triple = None
    
    return has_triple

Function was given 0.0 / 30 by autograder
Error detected in function. Displaying function below. 

def find_if_triplet(my_list):
    has_triple = None
    
    return has_triple

How many points? (max. 30): 0
-----
Could not import function. Displaying function below. 

def is_sum_square

How many points? (max. 

How many points? (max. 20): 10
-----
Error detected in function. Displaying function below. 

def find_if_triplet(my_list):
    has_triplet=True
    
    for index in my_list:
        counting=my_list.count(my_list[index])
        print(counting)
        if counting >=3:
            has_triplet
        else:
            has_triplet=False
           
    return has_triplet

print(find_if_triplet([1,1,1]))
print(find_if_triplet([1,2,3,3,3]))
print(find_if_triplet([1,2,2,2,2,3]))
print(find_if_triplet([1,2,2,3,4,4,5]))

How many points? (max. 30): 20
-----
Testing function below. 

def is_sum_squares(number):
    is_sq = True
    if (number**0.5)%1==0:
        is_sq
    else:
        is_sq=False
        
    return is_sq

Function was given 15.0 / 30 by autograder
Error detected in function. Displaying function below. 

def is_sum_squares(number):
    is_sq = True
    if (number**0.5)%1==0:
        is_sq
    else:
        is_sq=False
        
    return is_sq

How many points? (max. 30): 

How many points? (max. 10): 0
-----
Testing function below. 

def how_many_divide(number):
    counter = 0
    while (number>=2 and number%2==0):
        counter+=1
        number-=2**counter
    return counter
print(how_many_divide(3))
print(how_many_divide(4))
print(how_many_divide(12))
print(how_many_divide(64))

Function was given 10.0 / 20 by autograder
Error detected in function. Displaying function below. 

def how_many_divide(number):
    counter = 0
    while (number>=2 and number%2==0):
        counter+=1
        number-=2**counter
    return counter
print(how_many_divide(3))
print(how_many_divide(4))
print(how_many_divide(12))
print(how_many_divide(64))

How many points? (max. 20): 10
-----
Could not import function. Displaying function below. 

def find_if_triplet(my_list):
    has_triplet = None
    for i in my_list:
        if( [i]==[1] or [i]==[2] or 
    return has_triplet

print(find_if_triplet([1,1,1]))
print(find_if_triplet([1,2,3,3,3]))
print(find_if_triplet([1,2,2,

In [11]:
print(len(r.grades_list))

90


In [12]:
print(r.grades_list)

[('Jaron Ho', '1005011', 100.0), ('Colin Teoh', '1004989', 100.0), ('Tang Yi Wei Jennifer', '1005487', 100.0), ('Tham Jit', '1005036', 100.0), ('Chan Xinze', '1004990', 100.0), ('Lim Wei Kang Dylan', '1005496', 100.0), ('Ryan Toh', '1005129', 100.0), ('Lim Shu Hui Pamela', '1005647', 100.0), ('darren chow', '1005113', 100.0), ('Chng Sze Han', '1004991', 100.0), ('Issac Jose Ignatius', '1004999', 100.0), ('Low Ryan', '1005257', 100.0), ('Goh Ying Ming, Bryce', '1005016', 90.0), ('Aditya Sudarshan Rajesh Krishnan', '1005395', 60.0), ('Seow Sin Kiat', '1005022', 100.0), ('Michael Lim Kee Hian ', '1003427', 100.0), ('Zach Lim', '1002141', 92.5), ('Lek Jie Wei', '1005007', 100.0), ('Ishan Monnappa Kodira', '1005284', 100.0), ('Lim Rui-Chong Anthony', '1005264', 100.0), ('Constance Chua', '1005499', 70.0), ('Jade Megan Chan', '1005541', 90.0), ('Lee Chang Zheng', '1005289', 70.0), ('Lawrence Chen', '1005012', 100.0), ('Matthew Wong', '1005020', 100.0), ('Charlotte Ng', '1005320', 100.0), ('C

In [13]:
print(len(r.unique_grades_list))

89


In [14]:
print(r.unique_grades_list)

[('Jaron Ho', '1005011', 100.0), ('Colin Teoh', '1004989', 100.0), ('Tang Yi Wei Jennifer', '1005487', 100.0), ('Tham Jit', '1005036', 100.0), ('Chan Xinze', '1004990', 100.0), ('Lim Wei Kang Dylan', '1005496', 100.0), ('Ryan Toh', '1005129', 100.0), ('Lim Shu Hui Pamela', '1005647', 100.0), ('darren chow', '1005113', 100.0), ('Chng Sze Han', '1004991', 100.0), ('Issac Jose Ignatius', '1004999', 100.0), ('Low Ryan', '1005257', 100.0), ('Goh Ying Ming, Bryce', '1005016', 90.0), ('Aditya Sudarshan Rajesh Krishnan', '1005395', 60.0), ('Seow Sin Kiat', '1005022', 100.0), ('Michael Lim Kee Hian ', '1003427', 100.0), ('Zach Lim', '1002141', 92.5), ('Lek Jie Wei', '1005007', 100.0), ('Ishan Monnappa Kodira', '1005284', 100.0), ('Lim Rui-Chong Anthony', '1005264', 100.0), ('Constance Chua', '1005499', 70.0), ('Jade Megan Chan', '1005541', 90.0), ('Lee Chang Zheng', '1005289', 70.0), ('Lawrence Chen', '1005012', 100.0), ('Matthew Wong', '1005020', 100.0), ('Charlotte Ng', '1005320', 100.0), ('C

In [15]:
# To be executed to store custom grades to files
import pickle
save = False
if(save):
    with open("./pickle_A.pkl", "wb") as f:
        pickle.dump(r.unique_grades_list, f)
        f.close()